In [ ]:
# -*- coding: utf-8 -*-
import streamlit as st
import numpy as np
from catboost import CatBoostClassifier

# Tải mô hình CatBoost
@st.cache_resource
def load_model():
    model = CatBoostClassifier()
    model.load_model("catboost_asd.cbm")  # file .cbm bạn đã train
    return model

model = load_model()

st.set_page_config(page_title="Sàng Lọc Tự Kỷ", page_icon="🧠", layout="centered")
st.title("🔎 Ứng dụng Sàng Lọc Tự Kỷ (ASD)")

# Giao diện nhập dữ liệu
def nhap_du_lieu():
    st.header("📝 Vui lòng trả lời các câu hỏi sau:")

    # 10 câu hỏi A1 - A10
    questions = [
        "1. Có khi nào người được đánh giá tránh giao tiếp bằng mắt?",
        "2. Người đó có thích chơi một mình?",
        "3. Người đó có hay lặp lại từ/ngôn ngữ không?",
        "4. Người đó có khó khăn khi hiểu cảm xúc người khác?",
        "5. Có khi nào người đó không phản hồi khi được gọi tên?",
        "6. Người đó có nhạy cảm với âm thanh không?",
        "7. Có khi nào người đó không chia sẻ hứng thú hoặc thành tích với người khác?",
        "8. Người đó có hành vi lặp đi lặp lại không?",
        "9. Người đó có gặp khó khăn khi thay đổi thói quen hoặc môi trường không?",
        "10. Có khi nào người đó không hiểu các quy tắc xã hội cơ bản không?"
    ]
    a_scores = []
    for q in questions:
        ans = st.selectbox(q, ["Không", "Có"], key=q)
        a_scores.append(1 if ans == "Có" else 0)

    # Các đặc trưng khác
    age = st.number_input("11. Tuổi của người được đánh giá:", min_value=1, max_value=100, value=18)
    gender = st.selectbox("12. Giới tính:", ["Nam", "Nữ"])
    jundice = st.selectbox("13. Người đó có bị vàng da sau sinh không?", ["Không", "Có"])
    autism = st.selectbox("14. Gia đình bạn có người từng bị tự kỷ không?", ["Không", "Có"])
    used_app_before = st.selectbox("16. Bạn đã từng sử dụng ứng dụng này chưa?", ["Chưa", "Rồi"])
    relation = st.selectbox("15. Bạn là ai đối với người được đánh giá?",
                            ["Bố", "Mẹ", "Bản thân", "Người thân khác"])


    # Mã hóa dữ liệu
    gender_val = 1 if gender == "Nam" else 0
    jundice_val = 1 if jundice == "Có" else 0
    autism_val = 1 if autism == "Có" else 0
    used_app_val = 1 if used_app == "Rồi" else 0

    relation_map = {
        "Bố": [1, 0, 0, 0],
        "Mẹ": [0, 1, 0, 0],
        "Bản thân": [0, 0, 1, 0],
        "Người thân khác": [0, 0, 0, 1]
    }
    relation_val = relation_map[relation]

    # Gộp thành vector đặc trưng
    features = a_scores + [age, gender_val, jundice_val, autism_val, used_app_val] + relation_val
    return np.array([features])

# Hàm dự đoán
def du_doan(features):
    proba = model.predict_proba(features)[0][1]
    pred = int(proba >= 0.5)
    return pred, proba

# Chạy ứng dụng
input_data = nhap_du_lieu()
if st.button("📊 Dự đoán khả năng tự kỷ"):
    pred, proba = du_doan(input_data)

    st.subheader("🔍 Kết quả sàng lọc:")
    st.write(f"👉 Xác suất mắc tự kỷ (ASD): **{proba:.2f}**")

    st.markdown("### 🧭 Gợi ý hành động tiếp theo:")
    if pred == 1:
        st.error("""
        ⚠️ Nguy cơ cao mắc ASD.
        🔹 Hãy liên hệ chuyên gia tâm lý hoặc cơ sở y tế để được tư vấn kỹ lưỡng hơn.
        🔹 Ghi chép lại các biểu hiện thường gặp trong cuộc sống hàng ngày.
        🔹 Có thể tham khảo tài liệu về ASD từ WHO, CDC hoặc các trung tâm hỗ trợ trong nước.
        """)
    else:
        st.success("""
        ✅ Nguy cơ thấp mắc ASD.
        🔹 Bạn có thể yên tâm ở thời điểm hiện tại.
        🔹 Nếu vẫn còn băn khoăn, hãy trao đổi thêm với chuyên gia.
        """)
